In [1]:
using Pkg
Pkg.activate("CT")
using CT
using ProgressMeter

  Activating project at `~/CT_MPS_mini/CT`


In [2]:
Pkg.status()

Project CT v0.1.0
Status `~/CT_MPS_mini/CT/Project.toml`
  [c7e460c6] ArgParse v1.2.0
  [6e4b80f9] BenchmarkTools v1.6.0
  [f67ccb44] HDF5 v0.17.2
⌅ [9136182c] ITensors v0.6.19
  [682c06a0] JSON v0.21.4
  [33e6dc65] MKL v0.9.0
  [da04e1cc] MPI v0.20.23
  [3da0fdf6] MPIPreferences v0.1.11
⌅ [92933f4c] ProgressMeter v1.10.4
⌅ [9f0aa9f4] TCIITensorConversion v0.1.4
⌃ [b261b2ec] TensorCrossInterpolation v0.9.16
⌅ [a759f4b9] TimerOutputs v0.5.25
  [37e2e46d] LinearAlgebra
  [9a3f8284] Random
Info Packages marked with ⌃ and ⌅ have new versions available, but those with ⌅ are restricted by compatibility constraints from upgrading. To see why use `status --outdated`


In [3]:
include("global_adder_passthrough.jl")


adder_mpo_vec (generic function with 1 method)

In [4]:

function display_state(state)
    contracted = contract(state)
    state_vec = vec(Array(contracted, inds(contracted)...))
    # @show argmax(state_vec)
    loc = findall(x->abs(x)>1e-1, state_vec)
    # println(state_vec[loc])
    if length(loc) > 0
        pos = loc[1] - 1
        # ITensors native ordering: site 1 = rightmost bit in binary position
        # We reversed input, so reverse output to match our convention
        binary = string(pos, base=2, pad=L)
        return reverse(binary)  # site 1 = leftmost bit in our convention
    else
        return "0"^L
    end
end

display_state (generic function with 1 method)

In [ ]:
using Pkg
Pkg.activate("CT")
using CT
# benchmark adder_MPO
using ITensors
# initialize random state
using .CT: _initialize_basis, _initialize_vector, P_MPO, XI_MPO, I_MPO, adder_MPO, add1, power_mpo
using Random

L = 8
ancilla = 0
folded = true
seed_vec = 123457
xj = Set([1//3, 2//3])
i1 = 2
numerator = 1
denominator = 3
_maxdim = 2^(div(L,2))
_maxdim0 = 2^10
_eps = 1e-10
seed = 123457
x0 = nothing
qubit_site, ram_phy, phy_ram, phy_list = _initialize_basis(L, ancilla, folded)
rng = MersenneTwister(seed_vec)
rng_vec = seed_vec === nothing ? rng : MersenneTwister(seed_vec)
shift_1_3_bits, shift_1_3_amount = fraction_to_binary_shift(numerator, denominator, L)

  Activating project at `~/CT_MPS_mini/CT`


initializing random MPS with linkdims 1024


In [6]:
# initialize haining adder mpo
add1_mpo=MPO(add1(i1,L,phy_ram,phy_list),qubit_site);
add1_6,add1_3=power_mpo(add1_mpo,[div(2^L,6)+1,div(2^L,3)]);

In [7]:
# initialize the passthrough adder mpo
shift_bits, _ = fraction_to_binary_shift(numerator, denominator, L)
conn_pairs_dict, folded_ram_phy_dict = conn_pairs(L)
pairing_fixed_i1 = conn_pairs_dict[i1]
ram_phy_passthrough = folded_ram_phy_dict[i1]
# initialize the gate vector
gate_vec = initialize_gate_vec(pairing_fixed_i1, shift_bits, qubit_site, L, ram_phy_passthrough)

# sort the gate vector according to the qubit site number
gate_sites = [qubit_site_number(gate) for gate in gate_vec]
gate_vec_sorted = gate_vec[sortperm(gate_sites)];
gate_vec_collapsed = collapse_gate_vec(gate_vec_sorted);

# create the mpo
adder_mpo = create_mpo(gate_vec_collapsed);

In [14]:
for seed_vec in 1:10
    initial_state = _initialize_vector(L, ancilla, x0, folded, qubit_site, ram_phy, phy_ram, phy_list, rng_vec, _eps, _maxdim0);
    final_1 = apply(adder_mpo, initial_state; cutoff=_eps, maxdim=_maxdim);
    final_2 = apply(add1_3,initial_state; cutoff=_eps, maxdim=_maxdim);
    println(norm(final_1 - final_2))
end






initializing random MPS with linkdims 1024
2.870933999737604e-14
initializing random MPS with linkdims 1024
2.963260958235716e-14
initializing random MPS with linkdims 1024
3.116089685271521e-14
initializing random MPS with linkdims 1024
2.953704488170211e-14
initializing random MPS with linkdims 1024
2.661089605120238e-14
initializing random MPS with linkdims 1024
2.824006723298496e-14
initializing random MPS with linkdims 1024
3.066758753671839e-14
initializing random MPS with linkdims 1024
3.0916571756382026e-14
initializing random MPS with linkdims 1024
2.729150862397574e-14
initializing random MPS with linkdims 1024
2.765112915339986e-14


In [8]:
carry_links_passthrough, T_vec_passthrough, id_vec_passthrough, gate_vec_passthrough = initialize_links(L, qubit_site, shift_1_3_bits, ram_phy);
adder_mpo_passthrough = build_adder_mpo(qubit_site,L,carry_links_passthrough,gate_vec_passthrough, T_vec_passthrough,_eps,_maxdim);


In [9]:

dict_correct = Dict{String,String}()
for state_index in 1:2^L
    string_vec = lpad(string(state_index-1,base=2),L,"0")
    correct_unfolded = lpad(string((state_index - 1 + shift_1_3_amount) % 2^L,base=2),L,"0")
    string_vec_folded = join([string_vec[ram_phy[i]] for i in 1:L])
    correct_folded = join([correct_unfolded[ram_phy[i]] for i in 1:L])
    dict_correct[join(string_vec)] = join(correct_folded)
end


@showprogress for state_index in 1:2^L
    string_vec = lpad(string(state_index-1,base=2),L,"0")
    string_vec_folded = join([string_vec[ram_phy[i]] for i in 1:L])
    # Convention: RAM position i stores physical bit ram_phy[i]
    # productMPS assigns to RAM positions, so use folded representation
    vec = 1 .+ parse.(Int, [string(string_vec_folded[i]) for i in 1:L])
    initial_state = productMPS(qubit_site, vec)

    # @show CT.mps_element(initial_state, "000000001000"[ram_phy])
    final_state_1 = copy(initial_state)
    final_state_2 = copy(initial_state)
    # passthrough_state = copy(initial_state)
    final_state_1 = apply(adder_mpo, final_state_1; cutoff=_eps, maxdim=_maxdim);
    # passthrough_state = global_adder(passthrough_state, carry_links, T_vec, gate_vec, qubit_site; cutoff=_eps, maxdim=_maxdim);
    final_state_2 = apply(add1_3,final_state_2; cutoff=_eps, maxdim=_maxdim);
    # @show CT.mps_element(final_state_2, "001010110011"[ram_phy])
    # @show display_state(final_state_2)[phy_ram]
    final_state_1_unfolded = [ display_state(final_state_1)[phy_ram[i]] for i in 1:L ]
    final_state_2_unfolded = [ display_state(final_state_2)[phy_ram[i]] for i in 1:L ]
    # passthrough_state_unfolded = [ display_state(passthrough_state)[phy_ram[i]] for i in 1:L ]
    # println(string_vec, "=>", join(final_state_1_unfolded))
    # println(string_vec, "=>", join(final_state_2_unfolded))
    if join(final_state_1_unfolded) != join(final_state_2_unfolded)
        println(string_vec)
        println(join(final_state_1_unfolded))
        println(join(final_state_2_unfolded))
        # println(dict_correct[join(string_vec)])
        # println(join(passthrough_state_unfolded))
        println("wrong")
        # println("correct: ", dict_correct[join(string_vec)])

    end

    # println(string_vec, "=>", display_state(final_state_1), "<=>", display_state(final_state_2))
    # println(display_state(final_state_1) == display_state(final_state_2))
end

Progress: 100%|█████████████████████████████████████████| Time: 0:00:13
